<font size="5">**Project Summary**</font><br>
In this competition, the main task is to do surface time series classification. 1d convolution is a good technique when doing the time series classification job. So I apply 1d convolutional neural network to solve this problem. The whole code is written in Pytorch.

<font size="5">**External libs we are using**</font><br>

In [14]:
# libs we are using
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader

<font size="5">**Data preparations**</font><br>

In this project, I use the raw data as input of the network. I concatenated all datasets into one single numpy array. The first 3810 rows are training data with given labels, the rest 3816 rows are testing data where labels are not given. In total raw data has 7816 rows. In addition to that, we split the raw training data into training data and validation data by the ratio of 80/20.


I create three datasets and data loaders for them to make the data ready for model's training. The process is straightforward. I split the labelled data into two subsets, and keep testing data as is. Also, I convert Numpy arrays into `torch.tensor` objects of proper type (float for samples, and long - for targets).


In [15]:
def create_datasets(data, target, train_size, valid_pct=0.2, seed=None):
    """Converts NumPy arrays into PyTorch datsets.
    
    Three datasets are created in total:
        * training dataset
        * validation dataset
        * testing (un-labelled) dataset

    """
    raw =  data
    sz = train_size
    idx = np.arange(sz)
    trn_idx, val_idx = train_test_split(
        idx, test_size=valid_pct, random_state=seed)
     
    trn_ds = TensorDataset(
        torch.tensor(raw[:sz][trn_idx]).float(), 
        torch.tensor(target[:sz][trn_idx]).long())
    val_ds = TensorDataset(
        torch.tensor(raw[:sz][val_idx]).float(), 
        torch.tensor(target[:sz][val_idx]).long())
    tst_ds = TensorDataset(
        torch.tensor(raw[sz:]).float(), 
        torch.tensor(target[sz:]).long())
    return trn_ds, val_ds, tst_ds

# We use this function to divide the data
def create_datasets2(data, train_size, valid_pct=0.2, seed=None):
    """Converts NumPy arrays into PyTorch datsets.
    
    Three datasets are created in total:
        * training dataset
        * validation dataset
        * testing (un-labelled) dataset

    """
    raw =  data
    sz = train_size
  
    idx = np.arange(sz)
    print("total data length")
    print(len(idx))
    trn_idx, tst_idx = train_test_split(idx, test_size=0.2, random_state=1)
    trn_idx, val_idx = train_test_split(trn_idx, test_size=0.25, random_state=1)
    
    print("training data length")
    print(len(trn_idx))
    print("validation data length")
    print(len(val_idx))
    print("testing data length")
    print(len(tst_idx))
    
    trn_ds = TensorDataset(
        torch.tensor(raw[:sz][trn_idx]).float(), 
        torch.tensor(target[:sz][trn_idx]).long())
    val_ds = TensorDataset(
        torch.tensor(raw[:sz][val_idx]).float(), 
        torch.tensor(target[:sz][val_idx]).long())
    tst_ds = TensorDataset(
        torch.tensor(raw[:sz][tst_idx]).float(), 
        torch.tensor(target[:sz][tst_idx]).long())
    
    return trn_ds, val_ds, tst_ds

def create_loaders(data, bs=128, jobs=0):
    """Wraps the datasets returned by create_datasets function with data loaders."""
    
    trn_ds, val_ds, tst_ds = data
    trn_dl = DataLoader(trn_ds, batch_size=bs, shuffle=True, num_workers=jobs)
    val_dl = DataLoader(val_ds, batch_size=bs, shuffle=False, num_workers=jobs)
    tst_dl = DataLoader(tst_ds, batch_size=bs, shuffle=False, num_workers=jobs)
    return trn_dl, val_dl, tst_dl

<font size="5">**Neural Network Architecture**</font><br>

In [16]:
class Surface_Classifier(nn.Module):
    def __init__(self, raw_ni, no, drop=.5):
        super().__init__()
        
        self.conv1d = nn.Sequential(
            nn.Conv1d(raw_ni, 32, 8, 2, 3),
            nn.BatchNorm1d(32),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(    32,  64, 8, 4, 2),
            nn.BatchNorm1d(64),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(    64, 128, 8, 4, 2),
            nn.BatchNorm1d(128),
            nn.PReLU(),
            nn.Dropout(),
            nn.Conv1d(   128, 256, 8, 4, 2),
            nn.BatchNorm1d(256),
            nn.PReLU(),
            nn.Dropout()
        )
        
        self.dense = nn.Sequential(
            nn.Linear(256, 128), nn.ReLU(), nn.Dropout(),
            nn.Linear(128, 64), nn.Softmax(dim=1), 
            nn.Linear(64, no)
        )
            

    def forward(self, t_raw):
        conv1d_out = self.conv1d(t_raw)
        res = conv1d_out.view(conv1d_out.size(0), -1)        
        out = self.dense(res)
        return out

<font size="5">**Initial setup for random seed and project path**</font><br>

In [17]:

# set up the seed
seed = 1
np.random.seed(seed)

# path to sample submission
sample = Path.cwd().parent/'input'/'career-con-2019'/'sample_submission.csv'

ROOT = Path.cwd().parent/'input'/'career-con-2019-preprocessed-data'
raw_arr = np.load(ROOT/'feat.npy').transpose(0, 2, 1)
target = np.load(ROOT/'target.npy')

print("raw array shape")
print(raw_arr.shape)
print("label array shape")
print(target.shape)

trn_sz = 3810  # only the first `trn_sz` rows in each array include labelled data
#datasets = create_datasets((raw_arr), target, trn_sz, seed=seed)
datasets = create_datasets2((raw_arr), trn_sz, seed=seed)


# make sure that we run on a proper device (not relevant for Kaggle kernels but helpful in Jupyter sessions)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


trn_dl, val_dl, tst_dl = create_loaders(datasets, bs=256)

raw array shape
(7626, 9, 128)
label array shape
(7626,)
2286
762
762


<font size="5">**Training model**</font><br>
Now everything is ready to create a training loop and see if our model works. For each training epoch, the loop performs the following actions:
1. train model on the `trn_ds` dataset;
2. verify quality on the `val_ds` dataset;
3. check if the quality improved since previous epoch, and if so, save the model's weights onto disk;
4. in case if the model's quality isn't impoving for `patience` epochs, the training is stopped.
Also, the code tracks loss and accuracy history, and prints current scores with exponentially increasing logging frequency, i.e., only at 1, 2, 4, 8... epochs.

In [ ]:
raw_feat = raw_arr.shape[1]


lr = 0.001
n_epochs = 10000
iterations_per_epoch = len(trn_dl)
num_classes = 9
best_acc = 0
patience, trials = 500, 0
base = 1
step = 2
loss_history = []
acc_history = []

model = Surface_Classifier(raw_feat, num_classes).to(device)
criterion = nn.CrossEntropyLoss(reduction='sum')
opt = optim.Adam(model.parameters(), lr=lr)

print('Start model training')

for epoch in range(1, n_epochs + 1):
    
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(trn_dl):
        x_raw, y_batch = [t.to(device) for t in batch]
        opt.zero_grad()
        out = model(x_raw)
        loss = criterion(out, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        opt.step()
        
    epoch_loss /= trn_sz
    loss_history.append(epoch_loss)
    
    model.eval()
    correct, total = 0, 0
    for batch in val_dl:
        x_raw, y_batch = [t.to(device) for t in batch]
        out = model(x_raw)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_batch.size(0)
        correct += (preds == y_batch).sum().item()
    
    acc = correct / total
    acc_history.append(acc)

    if epoch % base == 0:
        print(f'Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}')
        base *= step
    print(f'Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}')

    
    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break
            
print('Done!')

Start model training
Epoch:   1. Loss: 1.3095. Acc.: 13.65%
Epoch:   1. Loss: 1.3095. Acc.: 13.65%
Epoch 1 best model saved with accuracy: 13.65%
Epoch:   2. Loss: 1.3030. Acc.: 21.78%
Epoch:   2. Loss: 1.3030. Acc.: 21.78%
Epoch 2 best model saved with accuracy: 21.78%
Epoch:   3. Loss: 1.2953. Acc.: 21.78%
Epoch:   4. Loss: 1.2853. Acc.: 21.78%
Epoch:   4. Loss: 1.2853. Acc.: 21.78%
Epoch:   5. Loss: 1.2728. Acc.: 29.79%
Epoch 5 best model saved with accuracy: 29.79%
Epoch:   6. Loss: 1.2605. Acc.: 29.66%
Epoch:   7. Loss: 1.2494. Acc.: 29.40%
Epoch:   8. Loss: 1.2405. Acc.: 29.66%
Epoch:   8. Loss: 1.2405. Acc.: 29.66%
Epoch:   9. Loss: 1.2323. Acc.: 29.92%
Epoch 9 best model saved with accuracy: 29.92%
Epoch:  10. Loss: 1.2242. Acc.: 30.58%
Epoch 10 best model saved with accuracy: 30.58%
Epoch:  11. Loss: 1.2163. Acc.: 31.23%
Epoch 11 best model saved with accuracy: 31.23%
Epoch:  12. Loss: 1.2083. Acc.: 32.41%
Epoch 12 best model saved with accuracy: 32.41%
Epoch:  13. Loss: 1.201

Epoch: 173. Loss: 0.8828. Acc.: 41.99%
Epoch: 174. Loss: 0.8830. Acc.: 42.78%
Epoch: 175. Loss: 0.8756. Acc.: 41.99%
Epoch: 176. Loss: 0.8803. Acc.: 42.13%
Epoch: 177. Loss: 0.8774. Acc.: 42.13%
Epoch: 178. Loss: 0.8734. Acc.: 42.91%
Epoch: 179. Loss: 0.8697. Acc.: 42.65%
Epoch: 180. Loss: 0.8702. Acc.: 42.39%
Epoch: 181. Loss: 0.8723. Acc.: 43.18%
Epoch: 182. Loss: 0.8646. Acc.: 42.65%
Epoch: 183. Loss: 0.8624. Acc.: 42.52%
Epoch: 184. Loss: 0.8780. Acc.: 43.44%
Epoch 184 best model saved with accuracy: 43.44%
Epoch: 185. Loss: 0.8672. Acc.: 42.65%
Epoch: 186. Loss: 0.8634. Acc.: 42.13%
Epoch: 187. Loss: 0.8677. Acc.: 43.04%
Epoch: 188. Loss: 0.8628. Acc.: 43.04%
Epoch: 189. Loss: 0.8578. Acc.: 42.26%
Epoch: 190. Loss: 0.8615. Acc.: 42.52%
Epoch: 191. Loss: 0.8602. Acc.: 42.52%
Epoch: 192. Loss: 0.8668. Acc.: 42.39%
Epoch: 193. Loss: 0.8571. Acc.: 42.13%
Epoch: 194. Loss: 0.8611. Acc.: 42.65%
Epoch: 195. Loss: 0.8542. Acc.: 42.39%
Epoch: 196. Loss: 0.8559. Acc.: 43.83%
Epoch 196 best 

Epoch: 367. Loss: 0.6979. Acc.: 50.52%
Epoch: 368. Loss: 0.6864. Acc.: 47.90%
Epoch: 369. Loss: 0.6999. Acc.: 48.03%
Epoch: 370. Loss: 0.6919. Acc.: 47.64%
Epoch: 371. Loss: 0.6879. Acc.: 49.87%
Epoch: 372. Loss: 0.6955. Acc.: 46.33%
Epoch: 373. Loss: 0.6918. Acc.: 49.87%
Epoch: 374. Loss: 0.6925. Acc.: 48.82%
Epoch: 375. Loss: 0.6854. Acc.: 46.72%
Epoch: 376. Loss: 0.6942. Acc.: 49.34%
Epoch: 377. Loss: 0.6807. Acc.: 49.34%
Epoch: 378. Loss: 0.6968. Acc.: 49.74%
Epoch: 379. Loss: 0.6788. Acc.: 48.95%
Epoch: 380. Loss: 0.6856. Acc.: 49.48%
Epoch: 381. Loss: 0.6893. Acc.: 49.08%
Epoch: 382. Loss: 0.6814. Acc.: 50.00%
Epoch: 383. Loss: 0.6793. Acc.: 48.56%
Epoch: 384. Loss: 0.6902. Acc.: 49.34%
Epoch: 385. Loss: 0.6813. Acc.: 47.90%
Epoch: 386. Loss: 0.6752. Acc.: 48.69%
Epoch: 387. Loss: 0.6658. Acc.: 50.79%
Epoch: 388. Loss: 0.6680. Acc.: 49.74%
Epoch: 389. Loss: 0.6729. Acc.: 49.34%
Epoch: 390. Loss: 0.6822. Acc.: 50.13%
Epoch: 391. Loss: 0.6666. Acc.: 49.48%
Epoch: 392. Loss: 0.6685.

Epoch: 559. Loss: 0.5568. Acc.: 56.82%
Epoch: 560. Loss: 0.5532. Acc.: 56.04%
Epoch: 561. Loss: 0.5528. Acc.: 58.01%
Epoch: 562. Loss: 0.5524. Acc.: 58.14%
Epoch: 563. Loss: 0.5490. Acc.: 57.74%
Epoch: 564. Loss: 0.5593. Acc.: 57.22%
Epoch: 565. Loss: 0.5558. Acc.: 57.48%
Epoch: 566. Loss: 0.5568. Acc.: 57.35%
Epoch: 567. Loss: 0.5582. Acc.: 56.17%
Epoch: 568. Loss: 0.5528. Acc.: 56.43%
Epoch: 569. Loss: 0.5514. Acc.: 57.74%
Epoch: 570. Loss: 0.5561. Acc.: 57.61%
Epoch: 571. Loss: 0.5535. Acc.: 57.87%
Epoch: 572. Loss: 0.5437. Acc.: 58.53%
Epoch 572 best model saved with accuracy: 58.53%
Epoch: 573. Loss: 0.5425. Acc.: 56.30%
Epoch: 574. Loss: 0.5549. Acc.: 56.82%
Epoch: 575. Loss: 0.5521. Acc.: 56.69%
Epoch: 576. Loss: 0.5508. Acc.: 56.04%
Epoch: 577. Loss: 0.5476. Acc.: 57.35%
Epoch: 578. Loss: 0.5598. Acc.: 57.35%
Epoch: 579. Loss: 0.5555. Acc.: 57.87%
Epoch: 580. Loss: 0.5508. Acc.: 58.14%
Epoch: 581. Loss: 0.5462. Acc.: 58.40%
Epoch: 582. Loss: 0.5440. Acc.: 57.48%
Epoch: 583. Los

Epoch: 756. Loss: 0.4355. Acc.: 64.96%
Epoch: 757. Loss: 0.4290. Acc.: 66.14%
Epoch: 758. Loss: 0.4124. Acc.: 65.09%
Epoch: 759. Loss: 0.4290. Acc.: 65.62%
Epoch: 760. Loss: 0.4261. Acc.: 66.01%
Epoch: 761. Loss: 0.4168. Acc.: 66.14%
Epoch: 762. Loss: 0.4229. Acc.: 65.09%
Epoch: 763. Loss: 0.4116. Acc.: 66.80%
Epoch: 764. Loss: 0.4195. Acc.: 66.27%
Epoch: 765. Loss: 0.4202. Acc.: 66.67%
Epoch: 766. Loss: 0.4142. Acc.: 66.40%
Epoch: 767. Loss: 0.4195. Acc.: 66.14%
Epoch: 768. Loss: 0.4201. Acc.: 67.19%
Epoch 768 best model saved with accuracy: 67.19%
Epoch: 769. Loss: 0.4140. Acc.: 66.14%
Epoch: 770. Loss: 0.4205. Acc.: 66.67%
Epoch: 771. Loss: 0.4083. Acc.: 66.27%
Epoch: 772. Loss: 0.4237. Acc.: 65.62%
Epoch: 773. Loss: 0.4115. Acc.: 67.32%
Epoch 773 best model saved with accuracy: 67.32%
Epoch: 774. Loss: 0.4186. Acc.: 66.40%
Epoch: 775. Loss: 0.4169. Acc.: 66.14%
Epoch: 776. Loss: 0.4092. Acc.: 66.14%
Epoch: 777. Loss: 0.4126. Acc.: 66.67%
Epoch: 778. Loss: 0.4121. Acc.: 65.49%
Epoch

Epoch: 959. Loss: 0.3731. Acc.: 66.80%
Epoch: 960. Loss: 0.3664. Acc.: 66.80%
Epoch: 961. Loss: 0.3449. Acc.: 66.14%
Epoch: 962. Loss: 0.3675. Acc.: 66.93%
Epoch: 963. Loss: 0.3588. Acc.: 67.06%
Epoch: 964. Loss: 0.3678. Acc.: 66.54%
Epoch: 965. Loss: 0.3544. Acc.: 66.93%
Epoch: 966. Loss: 0.3639. Acc.: 66.54%
Epoch: 967. Loss: 0.3509. Acc.: 66.67%
Epoch: 968. Loss: 0.3705. Acc.: 65.75%
Epoch: 969. Loss: 0.3772. Acc.: 65.88%
Epoch: 970. Loss: 0.3526. Acc.: 66.27%
Epoch: 971. Loss: 0.3840. Acc.: 66.93%
Epoch: 972. Loss: 0.3545. Acc.: 67.19%
Epoch: 973. Loss: 0.3716. Acc.: 67.45%
Epoch: 974. Loss: 0.3620. Acc.: 67.32%
Epoch: 975. Loss: 0.3634. Acc.: 66.40%
Epoch: 976. Loss: 0.3626. Acc.: 66.67%
Epoch: 977. Loss: 0.3714. Acc.: 67.98%
Epoch: 978. Loss: 0.3564. Acc.: 66.67%
Epoch: 979. Loss: 0.3646. Acc.: 66.01%
Epoch: 980. Loss: 0.3694. Acc.: 66.80%
Epoch: 981. Loss: 0.3519. Acc.: 67.59%
Epoch: 982. Loss: 0.3559. Acc.: 67.59%
Epoch: 983. Loss: 0.3651. Acc.: 65.62%
Epoch: 984. Loss: 0.3649.

Epoch: 1159. Loss: 0.3308. Acc.: 68.64%
Epoch: 1160. Loss: 0.3434. Acc.: 68.77%
Epoch: 1161. Loss: 0.3329. Acc.: 68.64%
Epoch: 1162. Loss: 0.3239. Acc.: 69.55%
Epoch: 1163. Loss: 0.3255. Acc.: 68.37%
Epoch: 1164. Loss: 0.3230. Acc.: 69.55%
Epoch: 1165. Loss: 0.3371. Acc.: 70.08%
Epoch: 1166. Loss: 0.3142. Acc.: 67.98%
Epoch: 1167. Loss: 0.3286. Acc.: 68.37%
Epoch: 1168. Loss: 0.3172. Acc.: 68.24%
Epoch: 1169. Loss: 0.3217. Acc.: 67.85%
Epoch: 1170. Loss: 0.3211. Acc.: 67.72%
Epoch: 1171. Loss: 0.3463. Acc.: 69.55%
Epoch: 1172. Loss: 0.3277. Acc.: 69.42%
Epoch: 1173. Loss: 0.3355. Acc.: 68.90%
Epoch: 1174. Loss: 0.3299. Acc.: 69.42%
Epoch: 1175. Loss: 0.3187. Acc.: 68.90%
Epoch: 1176. Loss: 0.3269. Acc.: 69.03%
Epoch: 1177. Loss: 0.3311. Acc.: 69.69%
Epoch: 1178. Loss: 0.3148. Acc.: 68.77%
Epoch: 1179. Loss: 0.3266. Acc.: 69.55%
Epoch: 1180. Loss: 0.3279. Acc.: 69.42%
Epoch: 1181. Loss: 0.3175. Acc.: 68.64%
Epoch: 1182. Loss: 0.3345. Acc.: 69.03%
Epoch: 1183. Loss: 0.3197. Acc.: 69.42%


<font size="5">**Compute result on testing data**</font><br>

In [ ]:
test_results = []
model.load_state_dict(torch.load('best.pth'))
model.eval()

# for batch in val_dl:
#     x_raw, y_batch = [t.to(device) for t in batch]
#     out = model(x_raw)
#     preds = F.log_softmax(out, dim=1).argmax(dim=1)
#     total += y_batch.size(0)
#     correct += (preds == y_batch).sum().item()
#     acc = correct / total
#     acc_history.append(acc)

# for batch in tst_dl:
#     x, y_batche = [t.to(device) for t in (batch)]
#     out = model(x)
#     y_hat = F.log_softmax(out, dim=1).argmax(dim=1)
#     test_results.extend(y_hat.tolist())
    
for batch in tst_dl:
        x_raw, y_batch = [t.to(device) for t in batch]
        out = model(x_raw)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_batch.size(0)
        correct += (preds == y_batch).sum().item()
    
acc = correct / total
print("accurancy on test data: "+str(acc))


<font size="5">**Store final result**</font><br>

In [ ]:
submit = pd.read_csv(sample)
enc = joblib.load(ROOT/'encoder.model')
submit['surface'] = enc.inverse_transform(test_results)
submit.to_csv('submit_base.csv', index=None)
print("store result successfully!")